# Boilerplate

In [ ]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time
import base64

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [ ]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [ ]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
CLIENT_PORT = 62012
SERVER_PORT = 62013

In [ ]:
client_tiface = await new_transport_interface("0.0.0.0", CLIENT_PORT)
client_tiface.run()

server_tiface = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface.run()

# Filter layer setup

In [ ]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

(client_fiface, server_fiface)

Find methods to run

In [ ]:
[m for m in dir(client_fiface) if not m.startswith('__')]

Run Client and Server Filter!

In [ ]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

## Filter layer operation

### As a client, connect to server

In [ ]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

In [ ]:
jupyter_endpoint = EndpointW(f"127.0.0.1:{SERVER_PORT}")

filter_cmd = FilterCmdW("sendrequestaction", endpoint=jupyter_endpoint, action=connect_action)
filter_cmd

In [ ]:
await client_fiface.command_response(filter_cmd)

In [ ]:
# XXX get endpoint programatically from client
client_endpoint = EndpointW(f"127.0.0.1:{CLIENT_PORT}")
client_endpoint

In [ ]:
server_notifications = server_fiface.get_notifications()
assert len(server_notifications) > 0
new_request_action = server_notifications[0]

assert new_request_action.request_action == connect_action

In [ ]:
cookie = base64.urlsafe_b64encode(bytes(new_request_action.request_action.name.encode()))
cookie

In [ ]:
response_ok = ResponseCodeW("loggedin", cookie=str(cookie), server_version="0.0.666")
response_ok

In [ ]:
await server_fiface.command_response(FilterCmdW("sendresponsecode", endpoint=client_endpoint, code=response_ok))

In [ ]:
client_notifications = client_fiface.get_notifications()
assert len(client_notifications) > 0
new_response_code = client_notifications[0]
new_response_code

In [ ]:
new_response_code.response_code

In [ ]:
cookie_from_server = new_response_code.response_code.cookie
assert str(cookie_from_server) == str(cookie)

### Convenience function

In [ ]:
async def request_action_to_filter(req_action, endpoint=jupyter_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await client_fiface.command_response(filter_cmd)

def get_all_notification(fiface, min_expected_count=1):
    notifications = fiface.get_notifications()
    assert len(notifications) >= min_expected_count
    
    return notifications

### Create a room

In [ ]:
await request_action_to_filter(RequestActionW("newroom", room_name="general"))

In [ ]:
get_all_notification(server_fiface)

### Join a room

In [ ]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

In [ ]:
get_all_notification(server_fiface)